    # Exploratory notebook for Bridge related operations
## Bitget API
### Trading
#### /api/mix/v1/order/placeOrder
Remarks :
If you try to close more than opened size, it will close the position nevertheless.

Request -------------------------------------------------------------------------------</br>
Place order doc : https://bitgetlimited.github.io/apidoc/en/mix/#place-order

productType

    umcbl USDT perpetual contract
    dmcbl Universal margin perpetual contract
    cmcbl USDC perpetual contract
    sumcbl USDT simulation perpetual contract
    sdmcbl Universal margin simulation perpetual contract
    scmcbl USDC simulation perpetual contract

candlestick interval

    1m(1minute)
    3m(3minute)
    5m(5minute)
    15m(15minute)
    30m(30minute)
    1H(1hour)
    2H (2hour)
    4H (4hour)
    6H (6hour)
    12H (12hour)
    1D (1day)
    3D (3day)
    1W(1week)
    1M (1month)
    6Hutc (UTC0 6hour)
    12Hutc (UTC0 12hour)
    1Dutc (UTC0 1day)
    3Dutc (UTC0 3day)
    1Wutc (UTC0 1 week)
    1Mutc (UTC0 1 month)

side

    open_long
    open_short
    close_long
    close_short
    buy_single buy under single_hold mode
    sell_single sell under single_hold mode

holdMode

    Position Mode
    Words 	Description
    single_hold 	One-way position
    double_hold 	Two-way position

orderType

    limit
    market

Response -------------------------------------------------------------------------------
</br>state

    init 	initial order, inserted into DB
    new 	new order, pending match in orderbook
    partially_filled 	Partially Filled
    filled 	Filled
    canceled 	Canceled

#### /api/mix/v1/singlePosition
Get single position from a symbol.

Remarks:
Will return both long and short info even if no pos is opened.
</br>
</br>

### Copy trading
#### Get Trader Open order
/api/mix/v1/trace/currentTrack
Limit rule 10 times/1s (uid).
Get current tracking ID to close the position as a trader.

Resp type :
```
{
    "code":"00000",
    "data":[
        {
            "symbol":"BTCUSDT_UMCBL",
            "trackingNo":"699848017573842632",
            "openOrderId":"4839999132328",
            "holdSide":"long",
            "openLeverage":20,
            "openAvgPrice":11451.5,
            "openTime":1602582690614,
            "openDealCount":"10",
            "stopProfitPrice":"123.14",
            "stopLossPrice":"20.52"
        }
    ],
    "msg":"success",
    "requestTime":1627354109502
}
```


#### Trader Close Position
/api/mix/v1/trace/closeTrackOrder
Limit rule 10 times/1s (uid)


## Runbot Webhooks
### Receiving webhooks
Whitelisting Runbot's IP</br>
Webhook format from Runbot :
```
{
  "exchange": "Binance" (Binance, Bitget, Bybit and DYDX),
  "market": "BTCUSDT" (or eth or bnb),
  "t": integer timestamp in ms,
  "positionType": "open" or "close" or "decrease" or "increase",
  "positionId": a string that identify the position, usefull in case where the webhook url is used for multiple runbot bots,
  "orderType": "market",
  "tradeDirection": "long" or "short" (it refers the order direction, no position direction, so for example "short" if "positionType" = "decrease" when this is a "long" position,
  "price": "28911.90",
  "size": "0.015"
  "leverage": "0.86736" if the current account have a bank roll of 500 USDT with that size and price. It refers to the leverage of the order size relative to the current total bankroll (it is usefull if you want to send % of bankroll size instead of fixed absolute size)
}

### Sending webhooks for testing
Strategy id 1 :
Open long :
``` bash
curl -X POST http://localhost:5001/runbot -H "Content-Type: application/json" -d '{
  "exchange": "Binance",
  "market": "BTCUSDT",
  "t": 1611192000000,
  "positionType": "open",
  "positionId": "1",
  "orderType": "market",
  "tradeDirection": "long",
  "price": "28911.90",
  "size": "0.01",
  "leverage": "0.86736"
}'
```
Increase long :
``` bash
curl -X POST http://localhost:5001/runbot -H "Content-Type: application/json" -d '{
  "exchange": "Binance",
  "market": "BTCUSDT",
  "t": 1611192000000,
  "positionType": "increase",
  "positionId": "1",
  "orderType": "market",
  "tradeDirection": "long",
  "price": "28911.90",
  "size": "0.01",
  "leverage": "0.86736"
}'
```

Decreasing long :
``` bash
curl -X POST http://localhost:5001/runbot -H "Content-Type: application/json" -d '{
  "exchange": "Binance",
  "market": "BTCUSDT",
  "t": 1611192000000,
  "positionType": "decrease",
  "positionId": "1",
  "orderType": "market",
  "tradeDirection": "long",
  "price": "28911.90",
  "size": "0.01",
  "leverage": "0.86736"
}'
```

Closing trade with strategy id 1:
``` bash
curl -X POST http://localhost:5001/runbot -H "Content-Type: application/json" -d '{
  "exchange": "Binance",
  "market": "BTCUSDT",
  "t": 1611192000000,
  "positionType": "close",
  "positionId": "1",
  "orderType": "market",
  "tradeDirection": "long",
  "price": "28911.90",
  "size": "0.01",
  "leverage": "0.86736"
}'
```




In [8]:
# Get symbols id for BTCUSDT
import requests
import json

resp = requests.get("https://api.bitget.com/api/mix/v1/market/contracts?productType=umcbl").json()
for symbol in resp['data']:
    if symbol['baseCoin'] == 'ETH':
        print(symbol['symbol'])

ETHUSDT_UMCBL


In [ ]:
# Test webhooks for runbot in staging/prod on thismavps
import requests
import json
